# Image Generator

In [1]:
#| default_exp image_generator

In [2]:
#| export
from Cloud2DImageConverter import spherical_projection as sp
from Cloud2DImageConverter import data
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import numpy as np
import pickle 
import shutil
import time
import os

Definição dos parâmetros da projeção. Os valores abaixo são padrão do respositório do semantic kitti api https://github.com/PRBonn/semantic-kitti-api/blob/master/auxiliary/laserscan.py

In [3]:
#| export
# FOV_UP = 3.0
# FOV_DOWN = -25.0
# WIDTH = 1024
# HEIGHT = 64
FOV_UP = 15
FOV_DOWN = -15
WIDTH = 1024
HEIGHT = 16

Definição de parâmetros deste notebook.
* FOLDER_PATH: Caminho contendo as sequências no formato do semantic_kitti
* BATCH_SIZE: Quantidade de frames que serão carregados por vez durante a projeção e geração das imagens, varia de acordo com a memória RAM disponível.

In [4]:
#| export
# FOLDER_PATH = "../point_clouds/semantic_kitti/"
FOLDER_PATH = "../point_clouds/carro/"
VELODYNE_PATH = FOLDER_PATH+"velodyne"
# LABEL_PATH = FOLDER_PATH+"labels"
LABEL_PATH = None
RESULTS_FOLDER =  "../results_carro/"

BATCH_SIZE = 500
MAX_LEN = len(os.listdir(VELODYNE_PATH))

if BATCH_SIZE > MAX_LEN: 
    BATCH_SIZE = MAX_LEN-1

In [5]:
#| export
if os.path.exists(RESULTS_FOLDER):
    shutil.rmtree(RESULTS_FOLDER)
    
os.makedirs(RESULTS_FOLDER)
os.makedirs(RESULTS_FOLDER+"/projections")
os.makedirs(RESULTS_FOLDER+"/labels")

In [6]:
#| export
def define_range(batch, batch_size, max_len):
    start = batch - batch_size
    end = batch
    if (max_len - batch) < batch_size:
        end = max_len
    return start, end

In [7]:
#| export
def do_projection(point_cloud):
    projection_dict = {"reflectance": [],"label": [], "depth": []}
    for points in point_cloud:
        reflectance, depth, mask = sp.spherical_projection(points, FOV_UP, FOV_DOWN, WIDTH, HEIGHT)
        projection_dict["reflectance"].append(reflectance)
        projection_dict["label"].append(mask)
        projection_dict["depth"].append(depth)
    return projection_dict

In [8]:
#| export
def save_image(matrix, save_path):
    img = Image.fromarray(matrix)
    final_path = os.path.join(RESULTS_FOLDER+save_path)
    file_number = len(os.listdir(final_path)) + 1
    file_name = f"{final_path}/{file_number:06d}.png"
    img.save(file_name)

def create_images(projection_dict):
    # Itera sobre cada posição do dicionário agrupando junto as matrizes de mesmo indice
    for zip_dict in zip(projection_dict["reflectance"], projection_dict["label"], projection_dict["depth"]):
        # Alterna entre as chaves e os indices respectivos a cada chave
        aux_dict = {}
        for index, key in enumerate(projection_dict.keys()):
            matrix = zip_dict[index]
            if key != "label":
                norm_matrix = ((matrix - matrix.min()) / (matrix.max() - matrix.min())) * 255
                matrix = norm_matrix.astype(np.uint8)
            else:
                matrix = sp.colored_matrix_with_label(matrix)
            aux_dict[key] = matrix
        # black_matrix = np.zeros((64, 1024), dtype=np.uint8)
        black_matrix = np.zeros((16, 1024), dtype=np.uint8)
        aux_dict["final_matrix"] = np.dstack((np.array(aux_dict["reflectance"]), np.array(aux_dict["depth"]), black_matrix))
        save_image(aux_dict["final_matrix"], "/projections")
        save_image(aux_dict["label"], "/labels")

In [9]:
#| export
label_list = None
for batch in tqdm(range(BATCH_SIZE, MAX_LEN, BATCH_SIZE), desc="Batch:"):
    # Define o intervalo de carregamento dos dados
    start, end = define_range(batch, BATCH_SIZE, MAX_LEN)
    # Define as listas contendo os nomes dos arquivos
    velodyne_list = sorted(os.listdir(VELODYNE_PATH))[start:end]
    if LABEL_PATH != None:
        label_list = sorted(os.listdir(LABEL_PATH))[start:end]
    # Carrega os dados 
    point_cloud = data.load_data(VELODYNE_PATH, velodyne_list, LABEL_PATH, label_list)
    # Realiza a projeção esférica
    projection_dict = do_projection(point_cloud)
    # Cria e salva as imagens
    create_images(projection_dict)

Batch:: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:36<00:00, 36.99s/it]


In [10]:
#| hide
import nbdev; nbdev.nbdev_export()